In [ ]:
from train.train_utils import train
from configs.configs_utils import update_config
from data.data_utils import (get_dataloader, get_distillation_dataloader)
from train.tools import (setup, setup_environ_flags, clear_gpu_cache)
from models.models_utils import  get_optimizer
from models.xrag_models_utils import get_xrag_models
from data.data_utils import get_dataloader
import torch

In [ ]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
train_config.batch_size_training = 4
train_config.distillation = True
train_config.num_workers_dataloader = 0
dataset_config.file = "data/loaders/squad-v2-sampled.py"

In [ ]:
rank = 0

In [ ]:
student_tokenizer, teacher_tokenizer, model = get_xrag_models()

In [ ]:
teacher_train_dataloader, teacher_eval_dataloader = get_dataloader(dataset_config, train_config, teacher_tokenizer, rank, distill_config)
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, student_tokenizer, rank, distill_config)

In [ ]:
optimizer = get_optimizer(model, train_config, fsdp_config)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=train_config.lr, epochs=train_config.num_epochs, steps_per_epoch=len(student_train_dataloader),
                                                pct_start=train_config.pct_start, div_factor=train_config.div_factor, final_div_factor=train_config.final_div_factor)

In [ ]:
results = train(
      model,
      student_train_dataloader,
      student_eval_dataloader,
      optimizer,
      scheduler,
      train_config.gradient_accumulation_steps,
      train_config,
      distill_config,
      dataset_config,
      teacher_train_dataloader if train_config.distillation else None,
      teacher_eval_dataloader if train_config.distillation else None,
      fsdp_config if train_config.enable_fsdp else None,
      None,
      rank,
  )
if rank == 0:
    [print(f'Key: {k}, Value: {v}') for k, v in results.items()]